In [1]:
from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json
load_dotenv(override=True)

python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 8


True

In [2]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [3]:
import os
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

openai = OpenAI(
    base_url=f"{endpoint}",
    api_key=api_key
)

In [4]:
todos = []
completed = []

In [5]:
def get_todo_report() -> str:
    result = ""
    for index, todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index + 1}: {todo}\n"
    show(result)
    return result

In [6]:
get_todo_report()

''

In [7]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [8]:
def mark_complete(index: int, completion_notes: str) -> str:
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [9]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish extra lab", "Eat banana"])

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: Buy groceries\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [10]:
mark_complete(1, "bought")

bought

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: [green][strike]Buy groceries[/strike][/green]\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [11]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [12]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [13]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [14]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [18]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-5.2-chat", messages=messages, tools=tools, reasoning_effort="none")
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)

In [25]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [26]:
todos, completed = [], []
loop(messages)

Todo #1: Define reference points and distances
Todo #2: Set up equations for positions over time
Todo #3: Solve for meeting time
Todo #4: Present final answer

Assume distance between Boston and New York is approximately 215 miles (reasonable estimate).

Todo #1: Define reference points and distances
Todo #2: Set up equations for positions over time
Todo #3: Solve for meeting time
Todo #4: Present final answer

Let t = hours after 2:00 pm. Boston train distance = 60t. New York train starts at t=1, distance = 80(t-1). Sum 
equals 215.

Todo #1: Define reference points and distances
Todo #2: Set up equations for positions over time
Todo #3: Solve for meeting time
Todo #4: Present final answer

Solve 60t + 80(t-1) = 215 → 140t = 295 → t ≈ 2.11 hours after 2:00 pm.

Todo #1: Define reference points and distances
Todo #2: Set up equations for positions over time
Todo #3: Solve for meeting time
Todo #4: Present final answer

Convert 0.11 hours to minutes (~7 minutes). Meeting time ≈ 4:07 pm.

Todo #1: Define reference points and distances
Todo #2: Set up equations for positions over time
Todo #3: Solve for meeting time
Todo #4: Present final answer

**✅ Solution**

• The distance between **Boston and New York** is reasonably estimated at **215 miles**.  
• Let **t** be the number of hours after **2:00 pm**.

**Train distances:**
- Boston train: travels at 60 mph → distance = **60t**
- New York train: starts at 3:00 pm (1 hour later), travels at 80 mph → distance = **80(t − 1)**

**Meeting condition:**
- Total distance = 215 miles  

  60t + 80(t − 1) = 215  

  140t = 295  

  t ≈ 2.11 hours after 2:00 pm

**Convert time:**
- 0.11 hours ≈ 7 minutes

### 🕓 **They meet at approximately 4:07 pm.**